In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings("ignore")
# I have used my google drive to load the training and testing data
# Upload the dataset file in your google drive and change the path to run the below line
data= pd.read_csv('/content/drive/MyDrive/ML/Priyadataset2c.csv',dtype='object')
print(data)

Mounted at /content/drive
                 height            diameter             weight  \
0      0.12493777936236    0.13190375787948   0.38616824929558   
1      0.14492137354649    0.12500746617994               0.75   
2     0.071892497830642                0.03    0.1631761874001   
3     0.084260612354435   0.037681879898677   0.23151165834924   
4                  0.07   0.090347950123449   0.19177699658224   
..                  ...                 ...                ...   
115    0.15324808700145     0.1301638325634   0.40118457408591   
116    0.12021522833926    0.11607358210855   0.33882777489585   
117    0.07556917126148   0.072067954099429   0.16160560001853   
118    0.13137129130651   0.091788907990009    0.2781412506351   
119    0.12825404991764    0.14820139358115   0.45533536592389   

                   hue      class  
0      2.9517669794008   Plastic   
1      3.4378755151121   Ceramic   
2      4.0521528606463   Plastic   
3      6.2831853071796   Ceramic   
4

In [2]:
X_train=data.iloc[:,:-1]
X_train=X_train.astype('float64')
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]
x_train = X.iloc[:,:].values
y_train = Y[:].values
X_train = X_train.to_numpy()
x_train=x_train.astype('float64')

In [3]:
x_train

array([[0.12493778, 0.13190376, 0.38616825, 2.95176698],
       [0.14492137, 0.12500747, 0.75      , 3.43787552],
       [0.0718925 , 0.03      , 0.16317619, 4.05215286],
       [0.08426061, 0.03768188, 0.23151166, 6.28318531],
       [0.07      , 0.09034795, 0.191777  , 2.12550385],
       [0.09979977, 0.10269434, 0.63616529, 3.53585184],
       [0.05      , 0.03      , 0.1       , 4.30117693],
       [0.07237162, 0.05452547, 0.12805399, 4.42275532],
       [0.07578485, 0.03      , 0.1       , 1.40192574],
       [0.0594732 , 0.06629474, 0.22178822, 2.32709256],
       [0.12586423, 0.12261042, 0.75      , 1.31546253],
       [0.16139965, 0.1262633 , 0.4311659 , 2.49095728],
       [0.07493345, 0.0302617 , 0.1       , 3.73277249],
       [0.0812203 , 0.0302167 , 0.14396059, 3.26946266],
       [0.11330191, 0.10137279, 0.48940241, 2.87357705],
       [0.09134435, 0.08960232, 0.37297987, 3.77208498],
       [0.07314564, 0.05168703, 0.21924605, 5.00764193],
       [0.12364714, 0.1164151 ,

In [4]:
for i in range(len(y_train)):
    if y_train[i] == ' Plastic ':
        y_train[i] = 0
    elif y_train[i] == ' Metal ':
        y_train[i] = 1
    elif y_train[i] == ' Ceramic ':
        y_train[i] = 2

In [5]:
def distance_linkage(L):
    if L == 0:
        return average
    elif L == 1:
        return single
    elif L == 2:
        return complete

In [6]:
def distance_calculation(a, b):
  dist=[]
  for ai , bi in zip(a,b):
    dist.append((ai - bi)**2)
  return math.sqrt(sum(dist))

In [7]:
def average(ai, aj):
  distances=[]
  for ci in ai:
    for cj in aj:
      distances.append(distance_calculation(ci, cj))
  return sum(distances) / len(distances)

In [8]:
def complete(ai, aj):
  distances=[]
  for ci in ai:
    for cj in aj:
      distances.append(distance_calculation(ci, cj))
  return max(distances)

In [9]:
def single(ai, aj):
  distances=[]
  for ci in ai:
    for cj in aj:
      distances.append(distance_calculation(ci, cj))
  return min(distances)

In [10]:
def initial_clusters(dataset):
    return {did: [dt] for did, dt in enumerate(dataset)}

In [11]:
def near_clusters(clusters,linkage):
        min_distance = math.inf
        near_clusters = None
        c_ids = list(clusters.keys())
        #print(c_ids)
        for i, cluster_i in enumerate(c_ids[:-1]):
            for j, cluster_j in enumerate(c_ids[i+1:]):
                dist = linkage(clusters[cluster_i], clusters[cluster_j])
                #print(dist)
                if dist < min_distance:
                    min_distance, near_clusters = dist, (cluster_i, cluster_j)
                    #print(near_clusters)
        return near_clusters

In [12]:
# merging into new clusters
def new_clusters(clusters, ci_id, cj_id):
        new_clusters = {0: clusters[ci_id] + clusters[cj_id]}
        #print(new_clusters)
        for clust_id in clusters.keys():
            if (clust_id == ci_id) | (clust_id == cj_id):
                continue
            new_clusters[len(new_clusters.keys())] = clusters[clust_id]
            #print(new_clusters)
        return new_clusters

In [13]:
def  AgglomerativeHierarchicalClustering(clusters,K,linkage):
        while len(clusters.keys()) > K:
            near_clusters1 = near_clusters(clusters,linkage)
            clusters = new_clusters(clusters,*near_clusters1)
        return clusters

In [14]:
def printw(clusters,X):
     y_pred=[]
     for id, points in clusters.items():
            #print("Cluster: {}".format(id))
            for point in points:
                #print("    {}".format(point))
                for i in X:
                  #print(i)
                  if((point == i).all()):
                    y_pred[i].append(id)
                    #print(y_pred)
     #print(y_pred)
     return y_pred

In [15]:
def prediction_Accuracy(x_test,y_pred,y_test):
  predit_sum = 0
  for i in range(len(x_test)):
    if(y_pred[i] == y_test[i]):
      predit_sum = predit_sum + 1
  return (predit_sum/len(x_test))*100

In [16]:
y_train =(np.array(y_train)).astype(int)

In [17]:
def two_cluster_accuracy(clusters, X):
    m_c1 = 0
    p_c1 = 0
    c_c1 = 0
    m_c2 = 0
    p_c2 = 0
    c_c2=0
  
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c1 += 1
                    elif (y_train[i] == 1):
                      m_c1 += 1
                    elif (y_train[i] ==2):
                      c_c1 += 1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c2 += 1
                    elif (y_train[i] == 1):
                      m_c2 += 1
                    elif (y_train[i] ==2):
                      c_c2 += 1
      
    c1_p = 0
    c2_p = 0
    if m_c1>p_c1 and m_c1>c_c1:
        c1_p = 1
    if p_c1>m_c1 and p_c1>c_c1:
      c1_p =0
    if c_c1>m_c1 and c_c1> p_c1:
      c1_p = 2
    if m_c2>p_c2 and m_c2>c_c2:
        c2_p = 1
    if p_c2>m_c2 and p_c2>c_c2:
      c2_p = 0
    if c_c2>m_c2 and c_c2> p_c2:
      c2_p = 2
    
    correct = 0
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c1_p):
                      correct +=1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c2_p):
                      correct +=1
    return (correct/100)*100.0
    

In [18]:
def four_cluster_accuracy(clusters, X):
    m_c1 = 0
    p_c1 = 0
    c_c1 = 0
    m_c2 = 0
    p_c2 = 0
    c_c2=0
    m_c3 = 0
    p_c3 = 0
    c_c3 = 0
    m_c4 = 0
    p_c4 = 0
    c_c4=0
  
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c1 += 1
                    elif (y_train[i] == 1):
                      m_c1 += 1
                    elif (y_train[i] ==2):
                      c_c1 += 1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c2 += 1
                    elif (y_train[i] == 1):
                      m_c2 += 1
                    elif (y_train[i] ==2):
                      c_c2 += 1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c3 += 1
                    elif (y_train[i] == 1):
                      m_c3 += 1
                    elif (y_train[i] ==2):
                      c_c3 += 1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c4 += 1
                    elif (y_train[i] == 1):
                      m_c4 += 1
                    elif (y_train[i] ==2):
                      c_c4 += 1
      
    c1_p = 0
    c2_p = 0
    c3_p =0
    c4_p =0
    if m_c1>p_c1 and m_c1>c_c1:
        c1_p = 1
    if p_c1>m_c1 and p_c1>c_c1:
      c1_p =0
    if c_c1>m_c1 and c_c1> p_c1:
      c1_p = 2
    if m_c2>p_c2 and m_c2>c_c2:
        c2_p = 1
    if p_c2>m_c2 and p_c2>c_c2:
      c2_p = 0
    if c_c2>m_c2 and c_c2> p_c2:
      c2_p = 2
    if m_c3>p_c3 and m_c3>c_c3:
        c3_p = 1
    if p_c3>m_c3 and p_c3>c_c3:
      c3_p = 0
    if c_c3>m_c3 and c_c3> p_c3:
      c3_p = 2
    if m_c4>p_c4 and m_c4>c_c4:
        c4_p = 1
    if p_c4>m_c4 and p_c4>c_c4:
      c4_p = 0
    if c_c4>m_c4 and c_c4> p_c4:
      c4_p = 2
    
    correct = 0
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c1_p):
                      correct +=1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c2_p):
                      correct +=1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c3_p):
                      correct +=1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c4_p):
                      correct +=1
    return (correct/100)*100.0
    

In [19]:
def six_cluster_accuracy(clusters, X):
    m_c1 = 0
    p_c1 = 0
    c_c1 = 0
    m_c2 = 0
    p_c2 = 0
    c_c2=0
    m_c3 = 0
    p_c3 = 0
    c_c3 = 0
    m_c4 = 0
    p_c4 = 0
    c_c4=0
    m_c5 = 0
    p_c5 = 0
    c_c5 = 0
    m_c6 = 0
    p_c6 = 0
    c_c6 = 0

  
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c1 += 1
                    elif (y_train[i] == 1):
                      m_c1 += 1
                    elif (y_train[i] ==2):
                      c_c1 += 1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c2 += 1
                    elif (y_train[i] == 1):
                      m_c2 += 1
                    elif (y_train[i] ==2):
                      c_c2 += 1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c3 += 1
                    elif (y_train[i] == 1):
                      m_c3 += 1
                    elif (y_train[i] ==2):
                      c_c3 += 1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c4 += 1
                    elif (y_train[i] == 1):
                      m_c4 += 1
                    elif (y_train[i] ==2):
                      c_c4 += 1
      if (id ==4):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c5 += 1
                    elif (y_train[i] == 1):
                      m_c5 += 1
                    elif (y_train[i] ==2):
                      c_c5 += 1
      if (id ==5):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c6 += 1
                    elif (y_train[i] == 1):
                      m_c6 += 1
                    elif (y_train[i] ==2):
                      c_c6 += 1
      
    c1_p = 0
    c2_p = 0
    c3_p =0
    c4_p =0
    c5_p =0
    c6_p =0
    if m_c1>p_c1 and m_c1>c_c1:
        c1_p = 1
    if p_c1>m_c1 and p_c1>c_c1:
      c1_p =0
    if c_c1>m_c1 and c_c1> p_c1:
      c1_p = 2
    if m_c2>p_c2 and m_c2>c_c2:
        c2_p = 1
    if p_c2>m_c2 and p_c2>c_c2:
      c2_p = 0
    if c_c2>m_c2 and c_c2> p_c2:
      c2_p = 2
    if m_c3>p_c3 and m_c3>c_c3:
        c3_p = 1
    if p_c3>m_c3 and p_c3>c_c3:
      c3_p = 0
    if c_c3>m_c3 and c_c3> p_c3:
      c3_p = 2
    if m_c4>p_c4 and m_c4>c_c4:
        c4_p = 1
    if p_c4>m_c4 and p_c4>c_c4:
      c4_p = 0
    if c_c4>m_c4 and c_c4> p_c4:
      c4_p = 2
    if m_c5>p_c5 and m_c5>c_c5:
        c5_p = 1
    if p_c5>m_c5 and p_c5>c_c5:
      c5_p = 0
    if c_c5>m_c5 and c_c5> p_c5:
      c5_p = 2
    if m_c6>p_c6 and m_c6>c_c6:
        c6_p = 1
    if p_c6>m_c6 and p_c6>c_c6:
      c6_p = 0
    if c_c6>m_c6 and c_c6> p_c6:
      c6_p = 2
    
    correct = 0
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c1_p):
                      correct +=1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c2_p):
                      correct +=1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c3_p):
                      correct +=1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c4_p):
                      correct +=1
      if (id ==4):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c5_p):
                      correct +=1
      if (id ==5):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c6_p):
                      correct +=1
    return (correct/100)*100.0
    

In [20]:
def Eight_cluster_accuracy(clusters, X):
    m_c1 = 0
    p_c1 = 0
    c_c1 = 0
    m_c2 = 0
    p_c2 = 0
    c_c2=0
    m_c3 = 0
    p_c3 = 0
    c_c3 = 0
    m_c4 = 0
    p_c4 = 0
    c_c4=0
    m_c5 = 0
    p_c5 = 0
    c_c5 = 0
    m_c6 = 0
    p_c6 = 0
    c_c6 = 0
    m_c7 = 0
    p_c7 = 0
    c_c7 = 0
    m_c8 = 0
    p_c8 = 0
    c_c8 = 0

  
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c1 += 1
                    elif (y_train[i] == 1):
                      m_c1 += 1
                    elif (y_train[i] ==2):
                      c_c1 += 1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c2 += 1
                    elif (y_train[i] == 1):
                      m_c2 += 1
                    elif (y_train[i] ==2):
                      c_c2 += 1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c3 += 1
                    elif (y_train[i] == 1):
                      m_c3 += 1
                    elif (y_train[i] ==2):
                      c_c3 += 1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c4 += 1
                    elif (y_train[i] == 1):
                      m_c4 += 1
                    elif (y_train[i] ==2):
                      c_c4 += 1
      if (id ==4):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c5 += 1
                    elif (y_train[i] == 1):
                      m_c5 += 1
                    elif (y_train[i] ==2):
                      c_c5 += 1
      if (id ==5):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c6 += 1
                    elif (y_train[i] == 1):
                      m_c6 += 1
                    elif (y_train[i] ==2):
                      c_c6 += 1
      if (id ==6):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c7 += 1
                    elif (y_train[i] == 1):
                      m_c7 += 1
                    elif (y_train[i] ==2):
                      c_c7 += 1

      if (id ==7):
        for ps in p:
          for i in range(len(X)):
                  #print(X[i])
                  if((ps == X[i]).all()):
                    if(y_train[i] == 0):
                      p_c8 += 1
                    elif (y_train[i] == 1):
                      m_c8 += 1
                    elif (y_train[i] ==2):
                      c_c8 += 1
      
    c1_p = 0
    c2_p = 0
    c3_p =0
    c4_p =0
    c5_p =0
    c6_p =0
    c7_p =0
    c8_p=0
    if m_c1>p_c1 and m_c1>c_c1:
        c1_p = 1
    if p_c1>m_c1 and p_c1>c_c1:
      c1_p =0
    if c_c1>m_c1 and c_c1> p_c1:
      c1_p = 2
    if m_c2>p_c2 and m_c2>c_c2:
        c2_p = 1
    if p_c2>m_c2 and p_c2>c_c2:
      c2_p = 0
    if c_c2>m_c2 and c_c2> p_c2:
      c2_p = 2
    if m_c3>p_c3 and m_c3>c_c3:
        c3_p = 1
    if p_c3>m_c3 and p_c3>c_c3:
      c3_p = 0
    if c_c3>m_c3 and c_c3> p_c3:
      c3_p = 2
    if m_c4>p_c4 and m_c4>c_c4:
        c4_p = 1
    if p_c4>m_c4 and p_c4>c_c4:
      c4_p = 0
    if c_c4>m_c4 and c_c4> p_c4:
      c4_p = 2
    if m_c5>p_c5 and m_c5>c_c5:
        c5_p = 1
    if p_c5>m_c5 and p_c5>c_c5:
      c5_p = 0
    if c_c5>m_c5 and c_c5> p_c5:
      c5_p = 2
    if m_c6>p_c6 and m_c6>c_c6:
        c6_p = 1
    if p_c6>m_c6 and p_c6>c_c6:
      c6_p = 0
    if c_c6>m_c6 and c_c6> p_c6:
      c6_p = 2
    if m_c7>p_c7 and m_c7>c_c7:
        c7_p = 1
    if p_c7>m_c7 and p_c7>c_c7:
      c7_p = 0
    if c_c7>m_c7 and c_c7> p_c7:
      c7_p = 2
    if m_c8>p_c8 and m_c8>c_c8:
        c8_p = 1
    if p_c8>m_c8 and p_c8>c_c8:
      c8_p = 0
    if c_c8>m_c8 and c_c8> p_c8:
      c8_p = 2
    
    correct = 0
    for id, p in clusters.items():
      if( id == 0):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c1_p):
                      correct +=1
      if (id ==1):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c2_p):
                      correct +=1
      if (id ==2):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c3_p):
                      correct +=1
      if (id ==3):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c4_p):
                      correct +=1
      if (id ==4):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c5_p):
                      correct +=1
      if (id ==5):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c6_p):
                      correct +=1
      if (id ==6):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c7_p):
                      correct +=1
      if (id ==7):
        for ps in p:
          for i in range(len(X)):
                  #print(i)
                  if((ps == X[i]).all()):
                    if(y_train[i] == c8_p):
                      correct +=1
    return (correct/100)*100.0
    

In [21]:
L =[ 0,1,2]
for l in L:
  K = 2
  print("For K = 2")
  if l == 0:
        print("Average linkage:")
  elif l == 1:
        print("Single linkage:")
  elif l == 2:
        print("Complete linkage:")
  clusters = initial_clusters(X_train)
  linkage1 = distance_linkage(l)
  final_clusters= AgglomerativeHierarchicalClustering(clusters,K,linkage1)
  print(two_cluster_accuracy(final_clusters,X_train))

For K = 2
Average linkage:
47.0
For K = 2
Single linkage:
45.0
For K = 2
Complete linkage:
49.0


In [22]:
L =[ 0,1,2]
for l in L:
  K = 4
  print("For K = 4")
  if l == 0:
        print("Average linkage:")
  elif l == 1:
        print("Single linkage:")
  elif l == 2:
        print("Complete linkage:")
  clusters = initial_clusters(X_train)
  linkage1 = distance_linkage(l)
  final_clusters= AgglomerativeHierarchicalClustering(clusters,K,linkage1)
  print(four_cluster_accuracy(final_clusters,X_train))

For K = 4
Average linkage:
49.0
For K = 4
Single linkage:
48.0
For K = 4
Complete linkage:
50.0


In [23]:
L =[ 0,1,2]
for l in L:
  K = 6
  print("For K = 6")
  if l == 0:
        print("Average linkage:")
  elif l == 1:
        print("Single linkage:")
  elif l == 2:
        print("Complete linkage:")
  clusters = initial_clusters(X_train)
  linkage1 = distance_linkage(l)
  final_clusters= AgglomerativeHierarchicalClustering(clusters,K,linkage1)
  print(six_cluster_accuracy(final_clusters,X_train))

For K = 6
Average linkage:
56.99999999999999
For K = 6
Single linkage:
49.0
For K = 6
Complete linkage:
50.0


In [24]:
L =[ 0,1,2]
for l in L:
  K = 8
  print("For K = 8")
  if l == 0:
        print("Average linkage:")
  elif l == 1:
        print("Single linkage:")
  elif l == 2:
        print("Complete linkage:")
  clusters = initial_clusters(X_train)
  linkage1 = distance_linkage(l)
  final_clusters= AgglomerativeHierarchicalClustering(clusters,K,linkage1)
  print(Eight_cluster_accuracy(final_clusters,X_train))

For K = 8
Average linkage:
56.99999999999999
For K = 8
Single linkage:
49.0
For K = 8
Complete linkage:
52.0


for 2 clusters --> complete linkage give best results

for 4 clusters --> complete linkage give best results

for 6 clusters --> average linkage give best results

for 8 clusters --> average linkage give best results